In [6]:
import tensorflow as tf
import numpy as np
from functools import partial
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib
import matplotlib.pyplot as plt

n_inputs = 28 * 28
n_hidden1 = 500
n_hidden2 = 500
n_hidden3 = 20
n_hidden4 = n_hidden2
n_hidden5 = n_hidden1
n_outputs = n_inputs

learning_rate = 0.001

initializer = tf.contrib.layers.variance_scaling_initializer()
my_dense_layer = partial(tf.layers.dense, activation=tf.nn.elu, 
                        kernel_initializer=initializer)

X = tf.placeholder(tf.float32, [None, n_inputs])
hidden1 = my_dense_layer(X, n_hidden1)
hidden2 = my_dense_layer(hidden1, n_hidden2)
hidden3_mean = my_dense_layer(hidden2, n_hidden3, activation=None)
hidden3_gamma = my_dense_layer(hidden2, n_hidden3, activation=None)
noise = tf.random_normal(tf.shape(hidden3_gamma), dtype=tf.float32)
hidden3 = hidden3_mean + tf.exp(0.5 * hidden3_gamma) * noise
hidden4 = my_dense_layer(hidden3, n_hidden4)
hidden5 = my_dense_layer(hidden4, n_hidden5)
logits = my_dense_layer(hidden5, n_outputs, activation=None)
outputs = tf.sigmoid(logits)

xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=X, logits=logits)
reconstruction_loss = tf.reduce_sum(xentropy)
latent_loss = 0.5 * tf.reduce_sum(
    tf.exp(hidden3_gamma) + tf.square(hidden3_mean) - 1 - hidden3_gamma)
loss = reconstruction_loss + latent_loss

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_digits = 60
n_epochs = 100
batch_size = 150

mnist = input_data.read_data_sets("/tmp/data/")

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch})
        loss_test = loss.eval(feed_dict={X: mnist.test.images})
        print(epoch, "Test loss:", loss_test)
    codings_rnd = np.random.normal(size=[n_digits, n_hidden3])
    outputs_val = outputs.eval(feed_dict={hidden3: codings_rnd})
    
plt.figure()
for iteration in range(n_digits):
    plt.subplot(n_digits // 10, 10, iteration + 1)
    plt.imshow(outputs_val[iteration].reshape(28,28), cmap="Greys", interpolation="nearest")
    plt.axis("off")
plt.savefig("../plots/ex_15_03.pdf")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0 Test loss: 1.21194e+06
1 Test loss: 1.14687e+06
2 Test loss: 1.11145e+06
3 Test loss: 1.09178e+06
4 Test loss: 1.08046e+06
5 Test loss: 1.08656e+06
6 Test loss: 1.06501e+06
7 Test loss: 1.05894e+06
8 Test loss: 1.05732e+06
9 Test loss: 1.04913e+06
10 Test loss: 1.04312e+06
11 Test loss: 1.03864e+06
12 Test loss: 1.03879e+06
13 Test loss: 1.03511e+06
14 Test loss: 1.03448e+06
15 Test loss: 1.03292e+06
16 Test loss: 1.02475e+06
17 Test loss: 1.02383e+06
18 Test loss: 1.02182e+06
19 Test loss: 1.02596e+06
20 Test loss: 1.01785e+06
21 Test loss: 1.01786e+06
22 Test loss: 1.01929e+06
23 Test loss: 1.01468e+06
24 Test loss: 1.01246e+06
25 Test loss: 1.01049e+06
26 Test loss: 1.00877e+06
27 Test loss: 1.00946e+06
28 Test loss: 1.01155e+06
29 Test loss: 1.00889e+06
30 Test loss: 1.01154e+06
31 Test loss: 